In [ ]:
from collections import namedtuple
import random

In [2]:
import pandas as pd

In [3]:
import orchid
import orchid.reference_origins as origins

In [4]:
import orchid.project_loader as opl

In [5]:
import toolz.curried as toolz

In [6]:
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
import UnitsNet

In [7]:
# Path name of whatever project you wish to load.
loader = opl.ProjectLoader(
    r'c:\src\Orchid.IntegrationTestData\Project-frankNstein_Montney_UTM13_METERS.ifrac')

In [8]:
# The returned value from `native_project` is . NET project. NOTE: each instance 
# of the `ProjectLoader` class loads **exactly one** .NET project. To load a 
# different project, you must create a new `ProjectLoader` instance.
native_project = loader.native_project()

In [9]:
all_wells = native_project.Wells.Items

In [10]:
# well_of_interest = all_wells[0]  # 'Hori_01'
well_of_interest = all_wells[1]  # 'Hori_02'
# well_of_interest = all_wells[3]  # 'Vert_01'

In [11]:
well_of_interest.Name

'Hori_02'

In [12]:
all_stages_for_well_of_interest = well_of_interest.Stages.Items

In [13]:
len(all_stages_for_well_of_interest)

29

In [14]:
all_stage_numbers = list(range((len(all_stages_for_well_of_interest))))
random.shuffle(all_stage_numbers)
all_stage_numbers

[10,
 5,
 8,
 15,
 27,
 2,
 7,
 14,
 11,
 25,
 9,
 1,
 18,
 3,
 6,
 13,
 20,
 21,
 12,
 17,
 28,
 24,
 23,
 19,
 26,
 16,
 4,
 0,
 22]

In [15]:
# Field names
BAKKEN = 'Bakken'
MONTNEY = 'Montney'

# Bakken Well names
DEMO_1H = 'Demo_1H'
DEMO_2H = 'Demo_2H'
DEMO_3H = 'Demo_3H'
DEMO_4H = 'Demo_4H'

# Montney Well names
HORI_01 = 'Hori_01'
HORI_02 = 'Hori_02'
HORI_03 = 'Hori_03'
VERT_01 = 'Vert_01'

# Test "names"
ADDITIONAL_BASIC = 'additional_basic'
BOTTOM_LOCATION = 'bottom_location'
CLUSTER_COUNT = 'cluster_count'
CLUSTER_LOCATION = 'cluster_location'
TOP_LOCATION = 'top_location'

In [16]:
# Remember that the `sampled_stage_numbers` are all **one less** than the stage numbers 
# in the feature itself (no stage zero (0)).
sampled_stage_numbers = {
    HORI_01: {
        ADDITIONAL_BASIC: [0, 14, 5, 10],
        BOTTOM_LOCATION: [0, 14, 8, 6],
        CLUSTER_COUNT: [0, 14, 5, 13],
        CLUSTER_LOCATION: [0, 14, 9, 1],
        TOP_LOCATION: [0, 14, 6, 9],
    },
    HORI_02: {
        ADDITIONAL_BASIC: [0, 28, 13, 21],
        BOTTOM_LOCATION: [0, 28, 3, 16],
        CLUSTER_COUNT: [0, 28, 13, 16],
        CLUSTER_LOCATION: [0, 28, 4, 16],
        TOP_LOCATION: [0, 28, 9, 25],
    },
}
# sampled_stage_numbers = [0, 1, 2, 3]  # 'Vert_01'

In [17]:
@toolz.curry
def basic_data_for_stage_of_well(w, s):
    return (MONTNEY, w.Name, s.DisplayStageNumber,
            s.DisplayNameWithoutWell, s.OrderOfCompletionOnWell,
            s.GlobalStageSequenceNumber, stage_type_str(s.StageType))
basic_data_for_well = basic_data_for_stage_of_well(well_of_interest)

@toolz.curry
def stage_type_str(stage_type):
    stage_type_str_map = {0: 'PlugAndPerf',
                          1: 'SlidingSleeve',
                          2: 'SinglePointEntry',
                          3: 'OpenHole'}
    return stage_type_str_map[stage_type]

@toolz.curry
def make_basic_item(n):
    basic_data = basic_data_for_well(all_stages_for_well_of_interest[n])
    return {
        'field': [basic_data[0]],
        'name': [basic_data[1]],
        'stage_no': [basic_data[2]],
        'name_without_well': [basic_data[3]],
        'order': [basic_data[4]],
        'global_seq_no': [basic_data[5]],
        'stage_type': [basic_data[6]],
    }

all_items = toolz.pipe(toolz.get_in([well_of_interest.Name, 'additional_basic'], sampled_stage_numbers),
                       toolz.map(make_basic_item),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)

,field,name,stage_no,name_without_well,order,global_seq_no,stage_type
0,Montney,Hori_02,1,Stage-1,0,1,PlugAndPerf
1,Montney,Hori_02,29,Stage-29,28,70,PlugAndPerf
2,Montney,Hori_02,14,Stage-14,13,43,PlugAndPerf
3,Montney,Hori_02,22,Stage-22,21,59,PlugAndPerf


In [18]:
all_frames = [rf for rf in iter(origins.WellReferenceFrameXy)]
random_frames = list(toolz.concat([all_frames] * 2))
random.shuffle(random_frames)
random_frames

[<WellReferenceFrameXy.PROJECT: 1>,
 <WellReferenceFrameXy.ABSOLUTE_STATE_PLANE: 0>,
 <WellReferenceFrameXy.WELL_HEAD: 2>,
 <WellReferenceFrameXy.PROJECT: 1>,
 <WellReferenceFrameXy.WELL_HEAD: 2>,
 <WellReferenceFrameXy.ABSOLUTE_STATE_PLANE: 0>]

In [19]:
sampled_frames = {
    HORI_01: {
        BOTTOM_LOCATION: [
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.WELL_HEAD,
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
        ],
        CLUSTER_LOCATION: [
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
            origins.WellReferenceFrameXy.WELL_HEAD,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
        ],
        TOP_LOCATION: [
            origins.WellReferenceFrameXy.WELL_HEAD,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
        ],
    },
    HORI_02: {
        BOTTOM_LOCATION: [
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.WELL_HEAD,
            origins.WellReferenceFrameXy.PROJECT,
        ],
        CLUSTER_LOCATION: [
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.WELL_HEAD,
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
            origins.WellReferenceFrameXy.PROJECT,
        ],
        TOP_LOCATION: [
            origins.WellReferenceFrameXy.WELL_HEAD,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
            origins.WellReferenceFrameXy.PROJECT,
        ],
    },
    VERT_01: {
        BOTTOM_LOCATION: [
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
            origins.WellReferenceFrameXy.ABSOLUTE_STATE_PLANE,
            origins.WellReferenceFrameXy.PROJECT,
            origins.WellReferenceFrameXy.WELL_HEAD,
        ],
        CLUSTER_LOCATION: [
            
        ],
        TOP_LOCATION: [
            
        ],
    },
}

In [20]:
SubsurfacePoint = namedtuple('SubsurfacePoint', ['x', 'y', 'depth'])

@toolz.curry
def subsurface_location_in_units(in_units, subsurface_location):
    x = subsurface_location.X.ToUnit(in_units)
    y = subsurface_location.Y.ToUnit(in_units)
    depth = subsurface_location.Depth.ToUnit(in_units)
    return SubsurfacePoint(x, y, depth)
subsurface_location_in_project_units = subsurface_location_in_units(native_project.ProjectUnits.LengthUnit)

def get_stage_bottom_location(reference_frame, stage):
    return stage.GetStageLocationBottom(reference_frame.value, DepthDatum.KellyBushing)

def get_stage_top_location(reference_frame, stage):
    return stage.GetStageLocationTop(reference_frame.value, DepthDatum.KellyBushing)

@toolz.curry
def stage_location(w, location_func, rf, s):
    subsurface_location = location_func(rf, s)
    x, y, depth = subsurface_location_in_project_units(subsurface_location)
    return ('montney', w.Name, s.DisplayStageNumber, rf.name,
            f'{x.Value:.2f} {UnitsNet.Length.GetAbbreviation(x.Unit)}',
            f'{y.Value:.2f} {UnitsNet.Length.GetAbbreviation(y.Unit)}',
            f'{depth.Value:.2f} {UnitsNet.Length.GetAbbreviation(depth.Unit)}')
location_for_well = stage_location(well_of_interest)

In [21]:
stage_bottom_location_for_well = location_for_well(get_stage_bottom_location)

@toolz.curry
def make_bottom_location_item(rf, n):
    location_data = stage_bottom_location_for_well(rf, all_stages_for_well_of_interest[n])
    return {
        'field': [location_data[0]],
        'name': [location_data[1]],
        'stage_no': [location_data[2]],
        'frame': [location_data[3]],
        'x': [location_data[4]],
        'y': [location_data[5]],
        'depth': [location_data[6]],
    }

all_items = toolz.pipe(zip(toolz.get_in([well_of_interest.Name, BOTTOM_LOCATION], sampled_frames),
                           toolz.get_in([well_of_interest.Name, BOTTOM_LOCATION], sampled_stage_numbers)),
                       toolz.map(lambda pair: make_bottom_location_item(*pair)),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)

,field,name,stage_no,frame,x,y,depth
0,montney,Hori_02,1,ABSOLUTE_STATE_PLANE,659041.61 m,6177489.54 m,2547.18 m
1,montney,Hori_02,29,PROJECT,-265.31 m,136.65 m,2535.20 m
2,montney,Hori_02,4,WELL_HEAD,2251.94 m,-1702.43 m,2545.77 m
3,montney,Hori_02,17,PROJECT,647.61 m,-565.58 m,2544.27 m


In [22]:
@toolz.curry
def cluster_count_for_stage_of_well(w, s):
    return ('bakken', w.Name, s.DisplayStageNumber,
            s.NumberOfClusters)
cluster_count_for_well = cluster_count_for_stage_of_well(well_of_interest)

@toolz.curry
def make_cluster_count(n):
    cluster_count_data = cluster_count_for_well(all_stages_for_well_of_interest[n])
    return {
        'field': [cluster_count_data[0]],
        'name': [cluster_count_data[1]],
        'stage_no': [cluster_count_data[2]],
        'cluster_count': [cluster_count_data[3]],
    }

all_items = toolz.pipe(toolz.get_in([well_of_interest.Name, CLUSTER_COUNT], sampled_stage_numbers),
                       toolz.map(make_cluster_count),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

cluster_count_df = pd.DataFrame(data)
cluster_count_df

,field,name,stage_no,cluster_count
0,bakken,Hori_02,1,1
1,bakken,Hori_02,29,1
2,bakken,Hori_02,14,1
3,bakken,Hori_02,17,1


In [23]:
shuffled_cluster_numbers = []
for cluster_count in cluster_count_df['cluster_count']:
    candidates = list(range(1, cluster_count + 1))
    random.shuffle(candidates)
    shuffled_cluster_numbers.append(candidates)
shuffled_cluster_numbers

[[1], [1], [1], [1]]

In [24]:
sampled_cluster_numbers = {
    HORI_01: [1, 1, 1, 1],
    HORI_02: [1, 1, 1, 1],
    HORI_03: [1, 1, 1, 1],
    VERT_01: [1, 1, 1, 1],
}

In [25]:
def get_stage_cluster_location(reference_frame, stage, cluster_no):
    return stage.GetStageLocationCluster(cluster_no, reference_frame.value, DepthDatum.KellyBushing)

@toolz.curry
def stage_cluster_location(w, location_func, rf, s, cluster_no):
    subsurface_location = location_func(rf, s, cluster_no)
    x, y, depth = subsurface_location_in_project_units(subsurface_location)
    return ('montney', w.Name, s.DisplayStageNumber, cluster_no, rf.name,
            f'{x.Value:.2f} {UnitsNet.Length.GetAbbreviation(x.Unit)}',
            f'{y.Value:.2f} {UnitsNet.Length.GetAbbreviation(y.Unit)}',
            f'{depth.Value:.2f} {UnitsNet.Length.GetAbbreviation(depth.Unit)}')
cluster_location_for_well = stage_cluster_location(well_of_interest)
stage_cluster_location_for_well = cluster_location_for_well(get_stage_cluster_location)

@toolz.curry
def make_cluster_location_item(rf, n, cluster_no):
    location_data = stage_cluster_location_for_well(rf, all_stages_for_well_of_interest[n], cluster_no)
    return {
        'field': [location_data[0]],
        'name': [location_data[1]],
        'stage_no': [location_data[2]],
        'cluster_no': [location_data[3]],
        'frame': [location_data[4]],
        'x': [location_data[5]],
        'y': [location_data[6]],
        'depth': [location_data[7]],
    }

all_items = toolz.pipe(zip(toolz.get_in([well_of_interest.Name, CLUSTER_LOCATION], sampled_frames),
                           toolz.get_in([well_of_interest.Name, CLUSTER_LOCATION], sampled_stage_numbers),
                           sampled_cluster_numbers[well_of_interest.Name],),
                       toolz.map(lambda triple: make_cluster_location_item(*triple)),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)

,field,name,stage_no,cluster_no,frame,x,y,depth
0,montney,Hori_02,1,1,PROJECT,1823.90 m,-1468.04 m,2547.17 m
1,montney,Hori_02,29,1,WELL_HEAD,320.78 m,-218.66 m,2534.98 m
2,montney,Hori_02,5,1,ABSOLUTE_STATE_PLANE,658733.50 m,6177723.22 m,2544.52 m
3,montney,Hori_02,17,1,PROJECT,619.24 m,-543.02 m,2544.19 m


In [26]:
stage_top_location_for_well = location_for_well(get_stage_top_location)

@toolz.curry
def make_top_location_item(rf, n):
    location_data = stage_top_location_for_well(rf, all_stages_for_well_of_interest[n])
    return {
        'field': [location_data[0]],
        'name': [location_data[1]],
        'stage_no': [location_data[2]],
        'frame': [location_data[3]],
        'x': [location_data[4]],
        'y': [location_data[5]],
        'depth': [location_data[6]],
    }

all_items = toolz.pipe(zip(toolz.get_in([well_of_interest.Name, TOP_LOCATION], sampled_frames),
                           toolz.get_in([well_of_interest.Name, TOP_LOCATION], sampled_stage_numbers)),
                       toolz.map(lambda pair: make_top_location_item(*pair)),
                       list)

def merge_values(values):
    return list(toolz.concat(values))

data = toolz.merge_with(merge_values,
                        *all_items)

pd.DataFrame(data)

,field,name,stage_no,frame,x,y,depth
0,montney,Hori_02,1,WELL_HEAD,2423.59 m,-1832.70 m,2547.25 m
1,montney,Hori_02,29,PROJECT,-322.66 m,180.15 m,2534.73 m
2,montney,Hori_02,10,ABSOLUTE_STATE_PLANE,658324.33 m,6178037.58 m,2544.78 m
3,montney,Hori_02,26,PROJECT,-91.15 m,8.58 m,2536.80 m
